## RNN(多対1), RNN(1対1), Bidirectional RNN, CNNによるテキスト分類
- 各ニューラルネットワークのアーキテクチャとそれらを用いた分類問題への対処の理解
- Embedding構造の理解

参考：
- [機械学習・深層学習による自然言語処理入門 ~scikit-learnとTensorFlowを使った実践プログラミング](https://www.amazon.co.jp/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%BB%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AB%E3%82%88%E3%82%8B%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%E5%85%A5%E9%96%80-scikit-learn%E3%81%A8TensorFlow%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9F%E5%AE%9F%E8%B7%B5%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0-Compass-Data-Science/dp/4839966605/)
- [09_text_classification.ipynb](https://colab.research.google.com/drive/1iAE2OUx7nFZzyrasJMYGf8fxd_fO80t7)

In [116]:
import string

#import gensim
import numpy as np
import pandas as pd
import tensorflow as tf
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, Embedding, SimpleRNN, LSTM, Conv1D, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.models import load_model, Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
maxlen = 250   # 1オブザベーション（トークン化されたtext）の最大長
num_words = 40000    # ユニーク単語数上限
num_label = 2

In [3]:
%%time
## データのロードとクリーニング、列選択（review_body、star_rating）

def filter_by_ascii_rate(text, threshold=0.9):
    ascii_letters = set(string.printable)
    rate = sum(c in ascii_letters for c in text) / len(text)
    return rate <= threshold

def load_dataset(filename, n=5000, state=6):
    df = pd.read_csv(filename, sep='\t')

    # Converts multi-class to binary-class.
    mapping = {1: 0, 2: 0, 4: 1, 5: 1}
    df = df[df.star_rating != 3]
    df.star_rating = df.star_rating.map(mapping)

    # extracts Japanese texts.
    is_jp = df.review_body.apply(filter_by_ascii_rate)
    df = df[is_jp]

    # sampling.
    df = df.sample(frac=1, random_state=state)  # shuffle
    grouped = df.groupby('star_rating')
    df = grouped.head(n=n)
    return df.review_body.values, df.star_rating.values

def clean_html(html, strip=False):
    soup = BeautifulSoup(html, 'html.parser')
    # タグ除去
    text = soup.get_text(strip=strip)
    return text

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz'
x, y = load_dataset(url)
x = [clean_html(text, strip=True) for text in x]

CPU times: user 12.4 s, sys: 642 ms, total: 13 s
Wall time: 25.7 s


In [4]:
## 作成されたデータ
df = pd.DataFrame({'review_body':x, 'star_rating':y})
print(df.shape)
df.head()

(10000, 2)


,review_body,star_rating
0,現在、地球温暖化の悪影響が、ここまで顕在化しているとは想像していませんでした。特に、このまま...,1
1,このアクション映画ほど、男気を感じたものはあったのだろうか。シンプル構成で時間をたっぷりと使...,1
2,このアプリを入れて以来、かなりお世話になりました。私の場合、PCで作成したデータや画像を出先...,0
3,取り出してさっと撮影することが必要な旅行用に不可欠だと思います。,1
4,Kindleで使用しています。複数のCloudが管理できたり、ワードやエクセルが使えたりと素...,1


In [5]:
x[:2]

['現在、地球温暖化の悪影響が、ここまで顕在化しているとは想像していませんでした。特に、このまま海面温度が上昇を続けると、早晩、南極・北極の氷が大規模に溶けることによって相当の範囲の陸地が海に没し、十億人単位での難民が発生するという事実には、本当に衝撃を受けました。ある人が、「人間は地球にすくう癌細胞のようである。増殖・破壊を続け、最終的には自らも寄生先の死によって滅ぶ運命にある」と言っていたことを思い出しました。一方、主人公であるゴア氏が、聴衆が数十人〜数百人程度の世界中の教室・会議場をまわって危機を訴えるという、地道な「草の根」の活動にも大変感銘を受けました。個人的には、ゴア氏には、ブッシュ氏との大統領戦における、「賢いが傲慢で冷徹」というイメージが強かったのですが、およそそんなことはない（＝きっと、ブッシュ陣営のネガティブ・キャンペーンの影響を愚かにも受けていたのでしょう）、信念を持ち、実行力が伴った、特筆すべき政治家であることも分かりました。温暖化の危機、ゴア氏ほかの活動家・学者の行動に対する不明を恥じると共に、自らもCO2ゼロ化に向けて早速行動したいと思いました。なお、当作品は、語られる事象のすべてが具体的かつ客観的に科学的根拠に支えられており、ドキュメンタリーとしても秀逸だと思います。是非ご覧になってみて下さい。お奨めします。',
 'このアクション映画ほど、男気を感じたものはあったのだろうか。シンプル構成で時間をたっぷりと使った中身のあるアクション映画です。劇中、鉄斧を持った闇のサイコ集団（敵）が出てくるんですがスタローンがこれでもかと、鉄拳制裁っ！するあたりはまさに圧巻の連続です。シンプルな映画ですが、家族で見ててハラハラドキドキします。また、この頃の映画って何か麻薬的な中毒性がありますよね。何度も映画の世界に引きずり込まれてしまいます。最近のアクション映画に物足りなさを感じている方、ぜひこのスタローンアクションを見て日々の疲れきった人生にスパイスを与えましょう。']

In [6]:
%%time
# トークン化
t = Tokenizer(wakati=True)

x = [' '.join(t.tokenize(text)) for text in x]

CPU times: user 2min 6s, sys: 227 ms, total: 2min 6s
Wall time: 2min 7s


In [7]:
x[:2]

['現在 、 地球 温暖 化 の 悪影響 が 、 ここ まで 顕在 化 し て いる と は 想像 し て い ませ ん でし た 。 特に 、 この まま 海面 温度 が 上昇 を 続ける と 、 早晩 、 南極 ・ 北極 の 氷 が 大 規模 に 溶ける こと によって 相当 の 範囲 の 陸地 が 海 に 没し 、 十 億 人 単位 で の 難民 が 発生 する という 事実 に は 、 本当に 衝撃 を 受け まし た 。 ある 人 が 、 「 人間 は 地球 に すくう 癌 細胞 の よう で ある 。 増殖 ・ 破壊 を 続け 、 最終 的 に は 自ら も 寄生 先 の 死 によって 滅ぶ 運命 に ある 」 と 言っ て い た こと を 思い出し まし た 。 一方 、 主人公 で ある ゴア 氏 が 、 聴衆 が 数 十 人 〜 数 百 人 程度 の 世界中 の 教室 ・ 会議 場 を まわっ て 危機 を 訴える という 、 地道 な 「 草の根 」 の 活動 に も 大変 感銘 を 受け まし た 。 個人 的 に は 、 ゴア 氏 に は 、 ブッシュ 氏 と の 大統領 戦 における 、 「 賢い が 傲慢 で 冷徹 」 という イメージ が 強かっ た の です が 、 およそ そんな こと は ない （ ＝ きっと 、 ブッシュ 陣営 の ネガティブ ・ キャンペーン の 影響 を 愚か に も 受け て い た の でしょ う ） 、 信念 を 持ち 、 実行 力 が 伴っ た 、 特筆 す べき 政治 家 で ある こと も 分かり まし た 。 温暖 化 の 危機 、 ゴア 氏 ほか の 活動 家 ・ 学者 の 行動 に対する 不明 を 恥じる と共に 、 自ら も CO 2 ゼロ 化 に 向け て 早速 行動 し たい と 思い まし た 。 なお 、 当 作品 は 、 語ら れる 事象 の すべて が 具体 的 かつ 客観 的 に 科学 的 根拠 に 支え られ て おり 、 ドキュメンタリー として も 秀逸 だ と 思い ます 。 是非 ご覧 に なっ て み て 下さい 。 お 奨め し ます 。',
 'この アクション 映画 ほど 、 男気 を 感じ た もの は あっ た の だろ う か 。 シンプル 構成 で

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(len(x_train), len(x_test), len(y_train), len(y_test))

8000 2000 8000 2000


***
#### tf.keras.preprocessing.text.Tokenizer
- Text tokenization utility class.
- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [9]:
test_texts = ['子ども たち に 見せ た ところ 、 もう 、 ページ を めくる たび に 面白い ねぇ という 喜び の 声 が 聞か れ まし た よ 。 今 の 子ども たち は 草原 を 通り抜け たり 、 川 を 渡っ たり など 、 お話 に 出 て くる よう な 体験 が あまり あり ませ ん 。 です から 、 この ポップアップ は ほんとに イメージ が 膨らん で くる ので いい なぁ と 思い ます 。 ぜひ 、 お 薦め です 。',
         '１ 枚 目 は 、 完全 に 演奏 は レイジ 、 メロディー は クリス と 、 水 と 油 みたい な 感じ で 違和感 たっぷり でし た が 、 ２ 枚 目 に なっ て 、 かなり クリス 色 の 強い バンド に 生まれ変わっ た 感じ が し ます 。 要は サウンド ガーデン っぽく なっ た 。 ただ 、 出し てる 音 は レイジ そのもの な ので 、 クリス の ボーカル スタイル に は 少し オケ が 寂しく 感じ て しまう 。 ７ 曲 目 くらい 、 ギター に も ボーカル に も エフェクター を 効か せ て 派手 に し て やっと クリスコーネル 節 が 生き て くる 。 スーパーアンノウン を 超える ほど の アルバム を 期待 し てる 私 として は また 肩透かし を 食らい まし た （ そりゃ 当人 たち は そんな 音楽 を する つもり は 無い の でしょ う が ） 冷静 に 、 サウンド ガーデン 、 レイジアゲインストザマシーン それぞれ の 一番 好き な アルバム と 比較 し て 、 それぞれ より も かっこいい と は 言え ませ ん 。 ただし 、 １ 枚 目 より ２ 枚 目 が 凄く 良い の は 確か な ので 、 ３ 枚 目 に 期待 を 込め て 、 この 作品 を 噛み締める という の が 、 レイジファン 、 クリス ファン の 楽しみ 方 な の かも しれ ませ ん 。 ８ 曲 目 とか １ ０ 曲 目 に 、 新しい 可能 性 を 感じ た 私 は そう し まし た 。']

tknzr = tf.keras.preprocessing.text.Tokenizer(num_words=100, oov_token='<UNK>')   # インプットの次元(単語の種類)を100とする場合
print(type(tknzr))
tknzr.fit_on_texts(test_texts)

<class 'keras_preprocessing.text.Tokenizer'>


In [10]:
# index : 単語
# 学習したデータは、全部で162語
# texts_to_sequencesを呼ぶ場合、num_words=100を上限として、それより後のindexの単語、もしくは存在しない単語は1: '<UNK>'に振り分けられる
tknzr.index_word

{1: '<UNK>',
 2: '、',
 3: 'に',
 4: 'は',
 5: 'を',
 6: 'の',
 7: '。',
 8: 'が',
 9: 'た',
 10: 'て',
 11: '目',
 12: 'な',
 13: 'と',
 14: '枚',
 15: 'し',
 16: 'クリス',
 17: '感じ',
 18: 'たち',
 19: 'まし',
 20: 'くる',
 21: 'ませ',
 22: 'ん',
 23: 'ので',
 24: '１',
 25: '曲',
 26: 'も',
 27: '子ども',
 28: 'という',
 29: 'たり',
 30: 'です',
 31: 'この',
 32: 'で',
 33: 'ます',
 34: 'レイジ',
 35: '２',
 36: 'なっ',
 37: 'サウンド',
 38: 'ガーデン',
 39: 'てる',
 40: 'ボーカル',
 41: 'アルバム',
 42: '期待',
 43: '私',
 44: 'それぞれ',
 45: 'より',
 46: '見せ',
 47: 'ところ',
 48: 'もう',
 49: 'ページ',
 50: 'めくる',
 51: 'たび',
 52: '面白い',
 53: 'ねぇ',
 54: '喜び',
 55: '声',
 56: '聞か',
 57: 'れ',
 58: 'よ',
 59: '今',
 60: '草原',
 61: '通り抜け',
 62: '川',
 63: '渡っ',
 64: 'など',
 65: 'お話',
 66: '出',
 67: 'よう',
 68: '体験',
 69: 'あまり',
 70: 'あり',
 71: 'から',
 72: 'ポップアップ',
 73: 'ほんとに',
 74: 'イメージ',
 75: '膨らん',
 76: 'いい',
 77: 'なぁ',
 78: '思い',
 79: 'ぜひ',
 80: 'お',
 81: '薦め',
 82: '完全',
 83: '演奏',
 84: 'メロディー',
 85: '水',
 86: '油',
 87: 'みたい',
 88: '違和感',
 89: 'たっぷり',
 90: 'でし',
 91: 'かなり

In [11]:
# Transforms each text in texts to a sequence of integers.
tknzr.texts_to_sequences(['体験 感じ 山 川 水', '曲 子ども abc 性 出し 音'])

# 1は<UNK>
# index=100の'音'は1となっている

[[68, 17, 1, 62, 85], [25, 27, 1, 1, 99, 1]]

***

In [12]:
# num_words=40000(インプットのベクトル次元)を上限とする辞書（indexと単語の対比）の作成と、単語ベースの学習データをindexベースへ変換
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<UNK>')
tokenizer

In [13]:
tokenizer.fit_on_texts(x_train)    # 学習データから辞書を作成

In [14]:
#tokenizer.word_index      # 辞書を表示すると長くなるので
i = 0
for key, value in tokenizer.word_index.items():
    print(key, ' : ' ,value)
    i += 1
    if i>10:
        break

<UNK>  :  1
の  :  2
、  :  3
。  :  4
が  :  5
は  :  6
に  :  7
て  :  8
た  :  9
を  :  10
で  :  11


In [15]:
# 学習データ（x_train）に存在した総単語数
len(tokenizer.word_index)

40410

In [18]:
x_train[:2]

['何一つ 本棚 に データー が 入っ て い ない 。 どう すれ ば 入る の か も 分から ない 。',
 '他 の かた の レビュー に も ある 様 に 十分 に 面白かっ た 。 カーペンター 版 ほど で は ない に せよ 何 度 も 観 返し たく なる 。 こう なっ たら ノルウェー と アメリカ の 生存 者 と で 続編 作っ ちゃっ て 欲しい な 。 思いっきり 絶望 的 な ラスト を 観 て み たい w']

In [19]:
x_train = tokenizer.texts_to_sequences(x_train)    # 単語ベースの学習データをindexベースへ変換
x_test = tokenizer.texts_to_sequences(x_test)      # 単語ベースのテストデータをindexベースへ変換

In [20]:
x_train[:1]

[[6981,
  13752,
  7,
  8239,
  5,
  293,
  8,
  23,
  17,
  4,
  185,
  559,
  59,
  840,
  2,
  20,
  13,
  526,
  17,
  4]]

In [31]:
x_train = pad_sequences(x_train, maxlen=maxlen, truncating='post', padding='post')        # これが学習に用いられる最終的なデータフォーマット
x_test = pad_sequences(x_test, maxlen=maxlen, truncating='post', padding='post')
# textがmaxlen=250より長い場合、後ろを打ち切る
# textがmaxlen=250より短い場合、後ろへパディング

In [32]:
x_train[:1]

array([[ 6981, 13752,     7,  8239,     5,   293,     8,    23,    17,
            4,   185,   559,    59,   840,     2,    20,    13,   526,
           17,     4,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [33]:
print(x_train.shape, x_test.shape)

(8000, 250) (2000, 250)


***
#### tf.keras.preprocessing.sequence.pad_sequences
- Pads sequences to the same length.
- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [24]:
# 例
pad_sequences([[68, 17, 1, 62], [25, 27, 1, 1, 99, 1]], maxlen=10)

array([[ 0,  0,  0,  0,  0,  0, 68, 17,  1, 62],
       [ 0,  0,  0,  0, 25, 27,  1,  1, 99,  1]], dtype=int32)

***
#### Embedding
- 正の整数（インデックス）を固定次元の密ベクトルに変換します．
- https://keras.io/ja/layers/embeddings/

In [25]:
test_input = np.array([[0, 1, 2, 3, 4],[5, 1, 2, 3, 6],[9, 9, 0, 1, 7]])
vocab_size = 10

In [26]:
test_model = Sequential()
test_model.add(Embedding(input_dim=vocab_size, output_dim=3, input_length=10))    # 3次元のEmbedding
test_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 3)             30        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


In [27]:
test_model.predict(test_input)

array([[[ 0.03483588,  0.02110745, -0.04418308],
        [ 0.03699956,  0.04006091, -0.0232033 ],
        [-0.01917415, -0.04058304,  0.02055681],
        [-0.00866901,  0.04949265, -0.04987855],
        [ 0.0324736 , -0.04071965, -0.0340901 ]],

       [[-0.04668638, -0.01326425, -0.04543719],
        [ 0.03699956,  0.04006091, -0.0232033 ],
        [-0.01917415, -0.04058304,  0.02055681],
        [-0.00866901,  0.04949265, -0.04987855],
        [-0.00274274, -0.01532656,  0.01879683]],

       [[ 0.03845752, -0.01631987, -0.01848371],
        [ 0.03845752, -0.01631987, -0.01848371],
        [ 0.03483588,  0.02110745, -0.04418308],
        [ 0.03699956,  0.04006091, -0.0232033 ],
        [ 0.00013831, -0.02045832,  0.02474436]]], dtype=float32)

In [28]:
print(test_input.shape, ' -> ', test_model.predict(test_input).shape)

# 3テキスト、各5単語 -> 各単語をoutput_dim=3次元に変換

(3, 5)  ->  (3, 5, 3)


***

## RNN

In [97]:
# モデル定義
#inp = Input(shape=(None,), name='input')
inp = Input(shape=(maxlen,), name='input')    # maxlen:文長（パディング込み単語数）
out = Embedding(input_dim=num_words, output_dim=300, mask_zero=True, trainable=True, name='embedding')(inp) # 各単語を300次元のベクトルに変換
out = SimpleRNN(units=100, return_sequences=False, name='rnn')(out)
out = Dense(units=num_label, activation='softmax')(out)
model_rnn = Model(inputs=inp, outputs=out)
model_rnn.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 250)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 250, 300)          12000000  
_________________________________________________________________
rnn (SimpleRNN)              (None, 100)               40100     
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 202       
Total params: 12,040,302
Trainable params: 12,040,302
Non-trainable params: 0
_________________________________________________________________


In [109]:
# in-out確認
print(x_test[:3].shape)

print(model_rnn(x_test[:3]).shape)
model_rnn(x_test[:3])

(3, 250)
(3, 2)


<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.617645  , 0.382355  ],
       [0.42916092, 0.57083905],
       [0.4959029 , 0.5040971 ]], dtype=float32)>

In [112]:
# コンパイル
model_rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [113]:
%%time
# 学習

batch_size = 128
epochs = 100
model_path = 'models/rnn'

model_rnn.fit(
    x=x_train, 
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[
        EarlyStopping(patience=3),
        ModelCheckpoint(model_path, save_best_only=True)
    ],
    shuffle=True
)

Epoch 1/100
50/50 [==============================] - 11s 208ms/step - loss: 0.6890 - acc: 0.5403 - val_loss: 0.6494 - val_acc: 0.6306
INFO:tensorflow:Assets written to: models/rnn/assets
Epoch 2/100
50/50 [==============================] - 10s 207ms/step - loss: 0.4140 - acc: 0.8610 - val_loss: 0.7027 - val_acc: 0.6438
Epoch 3/100
50/50 [==============================] - 10s 207ms/step - loss: 0.0530 - acc: 0.9889 - val_loss: 0.8681 - val_acc: 0.6100
Epoch 4/100
50/50 [==============================] - 11s 213ms/step - loss: 0.0045 - acc: 0.9999 - val_loss: 0.9437 - val_acc: 0.6131
CPU times: user 4min 12s, sys: 42.9 s, total: 4min 55s
Wall time: 43.7 s


In [95]:
# 予測
print(x_test.shape)
print('---------- sample input data ----------')
print(x_test[0])
print('--------------------------------------------')

y_pred = model_rnn.predict(x_test)
y_pred[:5]
# 結果：[prob(0), prob(1)]

(2000, 250)
---------- sample input data ----------
[2004    2  245    5 1106   16    5    3    1  109   74  319  361 7036
   14    8 4756   17   12   53   68  509    7   88   14   28    9    4
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0

array([[0.9139584 , 0.08604164],
       [0.06404482, 0.9359552 ],
       [0.162492  , 0.83750796],
       [0.00578979, 0.99421024],
       [0.97406524, 0.02593472]], dtype=float32)

In [94]:
# 精度評価
print('accuracy: {:.4f}'.format(accuracy_score(y_pred.argmax(axis=1), y_test)))
print('precision: {:.4f}'.format(precision_score(y_pred.argmax(axis=1), y_test)))
print('recall: {:.4f}'.format(recall_score(y_pred.argmax(axis=1), y_test)))
print('f1: {:.4f}'.format(f1_score(y_pred.argmax(axis=1), y_test)))

accuracy: 0.8065
precision: 0.7909
recall: 0.8193
f1: 0.8048


## RNN（1対1 アーキテクチャ）

In [150]:
# モデル定義
#inp = Input(shape=(None,), name='input')
inp = Input(shape=(maxlen,), name='input')    # maxlen:文長（パディング込み単語数）
out = Embedding(input_dim=num_words, output_dim=300, mask_zero=True, trainable=True, name='embedding')(inp) # 各単語を300次元のベクトルに変換
out = SimpleRNN(units=100, return_sequences=True, name='rnn')(out)   # 文で１つの出力でなく、文の各単語で１つの出力（各100次元）
out = GlobalMaxPooling1D()(out)    # 単語単位次元を文単位次元へ集約
out = Dense(units=num_label, activation='softmax')(out)
model_rnn1 = Model(inputs=inp, outputs=out)
model_rnn1.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 250)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 250, 300)          12000000  
_________________________________________________________________
rnn (SimpleRNN)              (None, 250, 100)          40100     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 100)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 202       
Total params: 12,040,302
Trainable params: 12,040,302
Non-trainable params: 0
_________________________________________________________________


In [151]:
# コンパイル
model_rnn1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [152]:
%%time
# 学習

batch_size = 128
epochs = 100
model_path = 'models/rnn1'

model_rnn1.fit(
    x=x_train, 
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[
        EarlyStopping(patience=3),
        ModelCheckpoint(model_path, save_best_only=True)
    ],
    shuffle=True
)

Epoch 1/100
50/50 [==============================] - 12s 235ms/step - loss: 0.7627 - acc: 0.5235 - val_loss: 0.6563 - val_acc: 0.6550
INFO:tensorflow:Assets written to: models/rnn1/assets
Epoch 2/100
50/50 [==============================] - 11s 227ms/step - loss: 0.5754 - acc: 0.7911 - val_loss: 0.7487 - val_acc: 0.6212
Epoch 3/100
50/50 [==============================] - 12s 237ms/step - loss: 0.4260 - acc: 0.8364 - val_loss: 0.5648 - val_acc: 0.7169
INFO:tensorflow:Assets written to: models/rnn1/assets
Epoch 4/100
50/50 [==============================] - 12s 233ms/step - loss: 0.2765 - acc: 0.9331 - val_loss: 0.5352 - val_acc: 0.7312
INFO:tensorflow:Assets written to: models/rnn1/assets
Epoch 5/100
50/50 [==============================] - 12s 234ms/step - loss: 0.1672 - acc: 0.9652 - val_loss: 0.5809 - val_acc: 0.7212
Epoch 6/100
50/50 [==============================] - 12s 239ms/step - loss: 0.0923 - acc: 0.9866 - val_loss: 0.6133 - val_acc: 0.7219
Epoch 7/100
50/50 [===============

In [153]:
# 予測

y_pred = model_rnn1.predict(x_test)
y_pred[:5]
# 結果：[prob(0), prob(1)]

array([[0.9970542 , 0.00294579],
       [0.04774555, 0.95225453],
       [0.07028291, 0.9297171 ],
       [0.1441599 , 0.85584015],
       [0.3739004 , 0.6260996 ]], dtype=float32)

In [154]:
# 精度評価
print('accuracy: {:.4f}'.format(accuracy_score(y_pred.argmax(axis=1), y_test)))
print('precision: {:.4f}'.format(precision_score(y_pred.argmax(axis=1), y_test)))
print('recall: {:.4f}'.format(recall_score(y_pred.argmax(axis=1), y_test)))
print('f1: {:.4f}'.format(f1_score(y_pred.argmax(axis=1), y_test)))

accuracy: 0.7360
precision: 0.7998
recall: 0.7123
f1: 0.7535


***
#### SimpleRNN、return_sequences=True/Falseの比較

In [155]:
batch = 1      #バッチサイズ
text_len = 10    # 文長（単語数）
emb_size = 100 # Enbedding size
data = np.random.randn(batch, text_len, emb_size)
data.shape

(1, 10, 100)

In [157]:
rnn1 = SimpleRNN(units=4, return_sequences=False)     # 多対１ アーキテクチャ

print(rnn1(data).shape)
rnn1(data)

(1, 4)


<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[-0.6500587 , -0.24879389, -0.6879822 , -0.99083513]],
      dtype=float32)>

In [160]:
rnn2 = SimpleRNN(units=4, return_sequences=True)     # １対１ アーキテクチャ

print(rnn2(data).shape)      # 文長のまま出力される
rnn2(data)

(1, 10, 4)


<tf.Tensor: shape=(1, 10, 4), dtype=float32, numpy=
array([[[ 0.76792735,  0.97077435,  0.95259386,  0.64597696],
        [ 0.6241702 ,  0.97440255, -0.3481571 ,  0.8570433 ],
        [ 0.6964981 ,  0.99988705,  0.77277637,  0.98311377],
        [ 0.8903857 ,  0.64649814,  0.59279853,  0.9962734 ],
        [ 0.80222005,  0.9726263 ,  0.9834009 , -0.9951626 ],
        [ 0.9782352 ,  0.9983396 ,  0.9591549 ,  0.55979997],
        [ 0.9923823 , -0.76988775, -0.75557476, -0.73030835],
        [ 0.3630569 ,  0.88714206,  0.5512719 , -0.89675605],
        [-0.9412261 ,  0.95178545,  0.975192  ,  0.6974632 ],
        [ 0.9130821 , -0.6712648 ,  0.3056602 ,  0.92335725]]],
      dtype=float32)>

***

## Bidirectional RNN

In [127]:
# モデル定義
#inp = Input(shape=(None,), name='input')
inp = Input(shape=(maxlen,), name='input')
out = Embedding(input_dim=num_words, output_dim=300, mask_zero=True, trainable=True, name='embedding')(inp) # 各単語を300次元のベクトルに変換
out = Bidirectional(SimpleRNN(units=100, return_sequences=False, name='rnn'))(out)
out = Dense(units=num_label, activation='softmax')(out)
model_rnn2 = Model(inputs=inp, outputs=out)
model_rnn2.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 250)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 250, 300)          12000000  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               80200     
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 402       
Total params: 12,080,602
Trainable params: 12,080,602
Non-trainable params: 0
_________________________________________________________________


In [128]:
# in-out確認
print(x_test[:3].shape)

print(model_rnn2(x_test[:3]).shape)
model_rnn2(x_test[:3])

(3, 250)
(3, 2)


<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.4475823 , 0.55241776],
       [0.54116607, 0.45883384],
       [0.4581206 , 0.54187936]], dtype=float32)>

In [129]:
# コンパイル
model_rnn2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [130]:
%%time
# 学習

batch_size = 128
epochs = 100
model_path = 'models/rnn2'

model_rnn2.fit(
    x=x_train, 
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[
        EarlyStopping(patience=3),
        ModelCheckpoint(model_path, save_best_only=True)
    ],
    shuffle=True
)

Epoch 1/100
50/50 [==============================] - 15s 283ms/step - loss: 0.6553 - acc: 0.6119 - val_loss: 0.5039 - val_acc: 0.7638
INFO:tensorflow:Assets written to: models/rnn2/assets
Epoch 2/100
50/50 [==============================] - 14s 285ms/step - loss: 0.2432 - acc: 0.9094 - val_loss: 0.5317 - val_acc: 0.7613
Epoch 3/100
50/50 [==============================] - 14s 284ms/step - loss: 0.0314 - acc: 0.9964 - val_loss: 0.6424 - val_acc: 0.7444
Epoch 4/100
50/50 [==============================] - 14s 275ms/step - loss: 0.0056 - acc: 0.9998 - val_loss: 0.5831 - val_acc: 0.7906
CPU times: user 7min 13s, sys: 1min 10s, total: 8min 23s
Wall time: 1min


In [133]:
# 予測

y_pred = model_rnn2.predict(x_test)
y_pred[:5]
# 結果：[prob(0), prob(1)]

array([[0.963963  , 0.03603706],
       [0.06148687, 0.93851316],
       [0.00653922, 0.9934608 ],
       [0.06518713, 0.9348129 ],
       [0.8471621 , 0.1528379 ]], dtype=float32)

In [135]:
# 精度評価
print('accuracy: {:.4f}'.format(accuracy_score(y_pred.argmax(axis=1), y_test)))
print('precision: {:.4f}'.format(precision_score(y_pred.argmax(axis=1), y_test)))
print('recall: {:.4f}'.format(recall_score(y_pred.argmax(axis=1), y_test)))
print('f1: {:.4f}'.format(f1_score(y_pred.argmax(axis=1), y_test)))

accuracy: 0.7955
precision: 0.8305
recall: 0.7788
f1: 0.8038


## CNN

In [143]:
# モデル定義
#inp = Input(shape=(None,), name='input')
inp = Input(shape=(maxlen,), name='input')
out = Embedding(input_dim=num_words, output_dim=300, mask_zero=False, trainable=True, name='embedding')(inp) # 各単語を300次元のベクトルに変換
out = Conv1D(filters=150, kernel_size=3, padding='valid', activation='relu', strides=1)(out)
out = GlobalMaxPooling1D()(out)    # 畳み込み後の文長（248）を1次元へMax Pooling
out = Dense(units=num_label, activation='softmax')(out)
model_cnn = Model(inputs=inp, outputs=out)
model_cnn.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 250)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 250, 300)          12000000  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 248, 150)          135150    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 150)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 302       
Total params: 12,135,452
Trainable params: 12,135,452
Non-trainable params: 0
_________________________________________________________________


In [144]:
# コンパイル
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [145]:
%%time
# 学習

batch_size = 128
epochs = 100
model_path = 'models/cnn'

model_cnn.fit(
    x=x_train, 
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[
        EarlyStopping(patience=3),
        ModelCheckpoint(model_path, save_best_only=True)
    ],
    shuffle=True
)

Epoch 1/100
50/50 [==============================] - 9s 178ms/step - loss: 0.6599 - acc: 0.6078 - val_loss: 0.5510 - val_acc: 0.7444
INFO:tensorflow:Assets written to: models/cnn/assets
Epoch 2/100
50/50 [==============================] - 8s 169ms/step - loss: 0.4549 - acc: 0.8291 - val_loss: 0.4418 - val_acc: 0.8006
INFO:tensorflow:Assets written to: models/cnn/assets
Epoch 3/100
50/50 [==============================] - 9s 172ms/step - loss: 0.2608 - acc: 0.9230 - val_loss: 0.3881 - val_acc: 0.8256
INFO:tensorflow:Assets written to: models/cnn/assets
Epoch 4/100
50/50 [==============================] - 8s 167ms/step - loss: 0.1234 - acc: 0.9784 - val_loss: 0.3837 - val_acc: 0.8388
INFO:tensorflow:Assets written to: models/cnn/assets
Epoch 5/100
50/50 [==============================] - 8s 168ms/step - loss: 0.0442 - acc: 0.9971 - val_loss: 0.3971 - val_acc: 0.8313
Epoch 6/100
50/50 [==============================] - 8s 167ms/step - loss: 0.0183 - acc: 0.9991 - val_loss: 0.4202 - val_ac

In [146]:
# 予測

y_pred = model_cnn.predict(x_test)
y_pred[:5]
# 結果：[prob(0), prob(1)]

array([[0.96953636, 0.03046361],
       [0.00173614, 0.9982639 ],
       [0.00543109, 0.99456894],
       [0.00274279, 0.9972573 ],
       [0.96378964, 0.03621037]], dtype=float32)

In [147]:
# 精度評価
print('accuracy: {:.4f}'.format(accuracy_score(y_pred.argmax(axis=1), y_test)))
print('precision: {:.4f}'.format(precision_score(y_pred.argmax(axis=1), y_test)))
print('recall: {:.4f}'.format(recall_score(y_pred.argmax(axis=1), y_test)))
print('f1: {:.4f}'.format(f1_score(y_pred.argmax(axis=1), y_test)))

accuracy: 0.8430
precision: 0.8434
recall: 0.8451
f1: 0.8442
